# Pipe Model

The `Pipe` class models the flow of water through a pipe. It uses the Darcy-Weisbach equation to calculate flow based on the pressure difference (head) between its ends. This notebook explains the `Pipe` model's functionality and provides a simulation example.

## State Variables

The state of the pipe is described by:

- `outflow` (float): The calculated flow rate through the pipe (m^3/s).
- `head_loss` (float): The loss of pressure head due to friction in the pipe (m).

## Parameters

The physical properties of the pipe are defined by:

- `length` (float): The length of the pipe (m).
- `diameter` (float): The diameter of the pipe (m).
- `friction_factor` (float): The Darcy friction factor for the pipe (dimensionless).

## Simulation Example

The following example simulates a pipe connecting two points with a varying head difference. The simulation demonstrates how the outflow changes in response to the changing upstream head. The results are plotted to visualize the relationship.

**Note:** To run this notebook, ensure you have `matplotlib` and `numpy` installed (`pip install matplotlib numpy`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the pipe
initial_state = State(outflow=0.0, head_loss=0.0)
parameters = Parameters(
    length=500.0,           # meters
    diameter=0.5,           # meters
    friction_factor=0.02    # dimensionless
)

# Create a Pipe instance
pipe = Pipe(name="main_pipe", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 1  # time step in seconds
simulation_duration = 100
num_steps = int(simulation_duration / dt)

# Store results for plotting
history = []

# Define external conditions
downstream_head = 50.0  # Constant downstream head (m)

# Run the simulation loop
for t in range(num_steps):
    # Vary the upstream head over time using a sine wave for demonstration
    upstream_head = 55.0 + 5.0 * np.sin(2 * np.pi * t / simulation_duration)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pipe.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", pipe.get_state())

# Prepare data for plotting
time_steps = [i * dt for i in range(num_steps)]
outflows = [s['outflow'] for s in history]
head_losses = [s['head_loss'] for s in history]
upstream_heads = [55.0 + 5.0 * np.sin(2 * np.pi * t / simulation_duration) for t in time_steps]

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_steps, outflows, label='Outflow')
ax1.set_ylabel('Outflow (m^3/s)')
ax1.set_title('Pipe Flow Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, upstream_heads, label='Upstream Head', linestyle='--')
ax2.axhline(downstream_head, color='r', linestyle='--', label='Downstream Head')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Head (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
